In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

################################################################################
## General Stuff
################################################################################
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote
from collection import collections
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from time import sleep
import time
from multiprocessing import Pool


################################################################################
## Music Stuff
################################################################################

## General
from discogsBase import discogs

## Discogs
from artist import artist
from artists import artists
from discogsUtils import discogsUtils
from albums import albums

## AllMusic
from artistAM import artistAM
from artistsAM import artistsAM
from discogsUtils import allmusicUtils
from artistsAM import artistsAM

## MusicBrainz
from artistMB import artistMB
from artistsMB import artistsMB
from discogsUtils import musicbrainzUtils



import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-03-15 10:36:50.937287


In [2]:
%load_ext autoreload
%autoreload
discAM   = discogs('allmusic')
artsAM   = artistsAM(discAM)
dutilsAM = allmusicUtils()

discDC   = discogs('discogs')
artsDC   = artists(discDC)
dutilsDC = discogsUtils()

discMB   = discogs('musicbrainz')
artsMB   = artistsMB(discMB)
dutilsMB = musicbrainzUtils()

_, _ = clock("Last Run")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Sun Mar 15, 2020 10:36:51 for Last Run


# Collections

In [ ]:
%load_ext autoreload
%autoreload
#col = collections(disc)
#col.parseCollections(force=False)
#col.createCollectionDBs()

# Parse Artists (All Music)

In [ ]:
for i in range(100):
    artsAM.parseArtistModValFiles(i, force=True)

In [16]:
artMB = artistMB(discMB)
retval = artMB.getData("/Volumes/Biggy/Discog/artists-musicbrainz/0/148150553282083715092252970758494088400.p")
retval.url.get()
retval.ID.get()

{'ID': 148150553282083715092252970758494088400, 'err': None}

# Parse Artists (Parallel)

In [3]:
%load_ext autoreload
%autoreload
discAM   = discogs('allmusic')
artsAM   = artistsAM(discAM)
dutilsAM = allmusicUtils()

discDC   = discogs('discogs')
artsDC   = artists(discDC)
dutilsDC = discogsUtils()

discMB   = discogs('musicbrainz')
artsMB   = artistsMB(discMB)
dutilsMB = musicbrainzUtils()

def parseArtistsAM(modVal, force=False, doExtra=False):
    artsAM.parseArtistModValFiles(modVal, force=force)

def parseArtistsDC(modVal, force=False, doExtra=False):
    artsDC.parseArtistModValFiles(modVal, force=force)

def parseArtistsMB(modVal, force=False, doExtra=False):
    artsMB.parseArtistModValFiles(modVal, force=force)
    
def parseArtistsParallel(db, nProcs=3, force=False):
    pool = Pool(processes=nProcs)
    if db == "Discogs":
        #result = pool.map_async(parseArtistsDC, [29, 30, 31, 32, 33, 34, 35])
        result = pool.map_async(parseArtistsDC, range(100))
    elif db == "AllMusic":
        result = pool.map_async(parseArtistsAM, range(100))
    elif db == "MusicBrainz":
        result = pool.map_async(parseArtistsMB, range(100))
    else:
        raise ValueError("[{0}] is not recognized as a DB".format(db))

    while not result.ready():
        if force is True:
            time.sleep(10)
        else:
            time.sleep(1)
    print("")
    return result.get()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
artsDC.parseArtistModValFiles(29, force=True)
#arts.parseArtistModValFiles(50, force=True)

Parsing Artist Files For ModVal 29
Forcing Reloads of ModVal=29
  Processing 7126 files.
	Processed 0/7126 files.
	Processed 500/7126 files.
	Processed 1000/7126 files.
	Processed 1500/7126 files.
	Processed 2000/7126 files.
	Processed 2500/7126 files.
	Processed 3000/7126 files.
	Processed 3500/7126 files.
	Processed 4000/7126 files.
	Processed 4500/7126 files.
	Processed 5000/7126 files.
	Processed 5500/7126 files.
	Processed 6000/7126 files.
	Processed 6500/7126 files.
	Processed 7000/7126 files.
Saving 7126 new artist IDs to /Volumes/Biggy/Discog/artists-discogs-db/29-DB.p
  --> This file is 12.0MB.
Saving 7126 new artist IDs name data to /Volumes/Biggy/Discog/artists-discogs-db/metadata/29-Metadata.p
  --> This file is 406.1kB.
Saving 7126 new artist IDs media data to /Volumes/Biggy/Discog/artists-discogs-db/metadata/29-MediaMetadata.p
  --> This file is 3.9MB.


7126

In [ ]:
start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")
#parseArtistsParallel(db="AllMusic", nProcs=3)
parseArtistsParallel(db="MusicBrainz", nProcs=3)
#parseArtistsParallel(db="Discogs", nProcs=3)
elapsed(start2, cmt2)
elapsed(start, cmt)

Current Time is Sun Mar 15, 2020 10:37:07 for Parsing All Artists
Current Time is Sun Mar 15, 2020 10:37:07 for Parallel Artist Parsing
Parsing Artist Files For ModVal 9
Parsing Artist Files For ModVal 0
Parsing Artist Files For ModVal 18
  ===> Found 525 files. There are 3 new files to parse.
  ===> Found 527 files. There are 6 new files to parse.
  ===> Found 553 files. There are 10 new files to parse.
Saving 325 new artist IDs to /Volumes/Biggy/Discog/artists-musicbrainz-db/18-DB.p
  --> This file is 135.3kB.
Saving 204 new artist IDs name data to /Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/18-Metadata.p
  --> This file is 17.8kB.
Saving 204 new artist IDs media data to /Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/18-MediaMetadata.p
  --> This file is 62.5kB.
Parsing Artist Files For ModVal 19
Saving 341 new artist IDs to /Volumes/Biggy/Discog/artists-musicbrainz-db/9-DB.p
  --> This file is 164.6kB.
Saving 185 new artist IDs name data to /Volumes/Biggy/Discog/art

In [3]:

from artists import artists
discDC   = discogs('discogs')
artsDC = artists(discDC)

from artistsAM import artistsAM
discAM   = discogs('allmusic')
artsAM = artistsAM(discAM)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Albums

In [ ]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
from discogsBase import discogs

disc = discogs()
albs = albums(disc)
#albs.parseAlbumModValFiles(modVal=27)

In [ ]:
def parseAlbums(modVal):
    albs.parseAlbumModValFiles(modVal=modVal)
    #albs.createAlbumModValMetadata(modVal)
    #albs.createAlbumModValMetadata(modVal=modVal)

def parseAlbumsParallel(modVals=None):
    if modVals is None:
        modVals = ["NAN"] + list(range(100))
        #modVals = range(100)
    nProcs = 2
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseAlbums, modVals)
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        #print(".",end="")
        time.sleep(10)
    print("")
    
    elapsed(start, cmt)
    return result.get()

In [ ]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
albs = albums(disc)

start,cmt=clock("Parsing All Albums")
start2,cmt2=clock("Parallel Album Parsing")
parseAlbumsParallel()
elapsed(start2, cmt2)
elapsed(start, cmt)

In [ ]:
from fsUtils import removeFile